In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

## Model

In [11]:
class Generator(nn.Module):
    def __init__(self, z_dim=100):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.ConvTranspose2d(z_dim, 512, 4, 1, 0, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),

            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),

            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),

            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias=False),
            nn.Tanh() 
        )
    def forward(self, z):
        output = self.model(z)
        return output


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(64, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(256, 512, 4, 2, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(512, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()  
        )
    def forward(self, img):
        output = self.model(img).view(-1, 1).squeeze(1)
        return output


In [ ]:
Discriminator = Discriminator()
Generator = Generator(100)

In [ ]:
Optimizer_Discriminator = optim.Adam(Discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))
Optimizer_Generator = optim.Adam(Generator.parameters(), lr=0.0002, betas=(0.5, 0.999))

In [ ]:
criterion = nn.BCELoss()

In [13]:
def train(Generator, Discriminator, criterion, train_loader, val_loader, epochs = 20, device = 'cpu'):
    for epoch in range(epochs):
        for batch in train_loader:
            Discriminator.train()
            Generator.eval()
            Discriminator.zero_grad()
            
            preds = Discriminator(batch.to(device))
            real_loss = criterion(preds, torch.ones_like(preds))
            real_loss.backward()
            
            fake_batch = Generator(torch.randn(batch.size(0), 100, 1, 1).to(device))
            fake_preds = Discriminator(fake_batch)
            fake_loss = criterion(fake_preds, torch.zeros_like(fake_preds))
            fake_loss.backward()
            
            Optimizer_Discriminator.step()
            
            Discriminator.eval()
            Generator.train()
            Generator.zero_grad()
            
            forged_batch = Generator(torch.randn(batch.size(0), 100, 1, 1).to(device))
            forged_preds = Discriminator(forged_batch)
            forged_loss = criterion(forged_preds, torch.ones_like(forged_preds))
            forged_loss.backward()
            
            Optimizer_Generator.step()
        
        print(f"Epoch: {epoch+1}/{epochs} | Real Loss: {real_loss.item()} | Fake Loss: {fake_loss.item()} | Forged Loss: {forged_loss.item()}")
